<a href="https://colab.research.google.com/github/ishwarvenugopal/ML-DL_Implementation/blob/master/Deep_Learning/MultiLayerPerceptron_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source: https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/

In [102]:
import torch
print(torch.__version__)

1.5.1+cu101


### Data Pre-processing: 
#### In PyTorch all the preprocessing operations need to be done inside a custom class derived from the Dataset class, so as to fit in the DataLoader function of PyTorch

In [107]:
from torch.utils.data import Dataset
from torch.utils.data import random_split

In [108]:
class CSVDataset(Dataset):
  #loading the dataset
  def __init__ (self, path):
    df = pd.read_csv(path)
    self.X = df.values[:,:-1]
    self.y = df.values[:,-1]
    self.X = self.X.astype('float32')
    self.y = LabelEncoder().fit_transform(self.y)
    self.y = self.y.astype('float32')
    self.y = self.y.reshape((len(self.y),1))

  #get the number of rows in the dataset
  def __len__(self):
    return len(self.X)

  #get a row at a particular index in the dataset
  def __getitem__ (self,idx):
    return [self.X[idx],self.y[idx]]
  
  # get the indices for the train and test rows
  def get_splits(self, n_test = 0.33):
    test_size = round(n_test * len(self.X))
    train_size = len(self.X) - test_size
    return random_split(self, [train_size, test_size])  

In [109]:
from torch.utils.data import DataLoader

In [110]:
def prepare_data(path):
  dataset = CSVDataset(path)
  train, test = dataset.get_splits()
  #print("Train: {}, Test: {}".format(train, test))
  # Prepare iterable data loader for train and test
  train_dl = DataLoader(train, batch_size=32, shuffle = True)
  test_dl = DataLoader(test, batch_size = 1024, shuffle = False)
  return train_dl, test_dl

In [111]:
# # For CNNs using images (eg.MNIST), following function can be used to prepar the data

# def prepare_data(path):
#     # define standardization
#     trans = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
#     # load dataset
#     train = MNIST(path, train=True, download=True, transform=trans)
#     test = MNIST(path, train=False, download=True, transform=trans)
#     # prepare data loaders
#     train_dl = DataLoader(train, batch_size=64, shuffle=True)
#     test_dl = DataLoader(test, batch_size=1024, shuffle=False)
#     return train_dl, test_dl

# # This function can then be called as: 

# from torch.utils.data import DataLoader
# from torchvision.datasets import MNIST
# from torchvision.transforms import Compose
# from torchvision.transforms import ToTensor
# from matplotlib.pyplot as plt

# path = '~/.torch/datasets/mnist'
# train_dl, test_dl = prepare_data(path)
# # Plotting from one batch
# i, (inputs, targets) = next(enumerate(train_dl))
# for i in range(25):
# 	plt.subplot(5, 5, i+1)
# 	plt.imshow(inputs[i][0], cmap='gray')
# plt.show()

In [112]:
# Data Preparation for training and testing

path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print("Train size: {}, Test size: {}".format(len(train_dl.dataset),len(test_dl.dataset)))

Train size: 234, Test size: 116


### Model Definition:
#### A custom class derived from the 'Module' class need to be defined to specify the model

In [72]:
from torch.nn import Module
from torch.nn import Linear,ReLU, Sigmoid
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [113]:
# Define a class for a Multi-Layer Perceptron model

class MLP(Module):
  #Defining the model elements
  def __init__(self,n_inputs):
    super(MLP,self).__init__()
    self.hidden1 = Linear(n_inputs, 10)
    kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
    self.act1 = ReLU()
    self.hidden2 = Linear(10,8)
    kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
    self.act2 = ReLU()
    self.hidden3 = Linear(8,1)
    xavier_uniform_(self.hidden3.weight)
    self.act3 = Sigmoid()
    #For multi-class (eg.3) problems, last layer will be as follows:
      # self.hidden3 = Linear(8, 3)
      # xavier_uniform_(self.hidden3.weight)
      # self.act3 = Softmax(dim=1)

  #Defining a forward propagation unit
  def forward(self,X):
    X = self.hidden1(X)
    X = self.act1(X)
    X = self.hidden2(X)
    X = self.act2(X)
    X = self.hidden3(X)
    X = self.act3(X)
    return X

In [114]:
model = MLP(34) # Create a model

### Training the model

In [115]:
from torch.nn import BCELoss
from torch.optim import SGD

In [116]:
# Training the model

def train_model(train_dl, test_dl):
  criterion = BCELoss()
  optimizer = SGD(model.parameters(), lr=0.01, momentum = 0.9)
  #enumerate epochs
  for epoch in range(100):
    #enumerate in mini batches
    for i, (inputs,targets) in enumerate(train_dl):
      optimizer.zero_grad() # Clearing the gradients
      yhat = model(inputs)
      loss = criterion(yhat,targets)
      loss.backward()
      optimizer.step() #Update backward weights

In [117]:
train_model(train_dl,test_dl)

### Evaluating the model

In [118]:
from numpy import vstack
from sklearn.metrics import accuracy_score

In [119]:
# Evaluating the model

def evaluate_model(test_dl, model):
  predictions, actuals = list(),list()
  for i,(inputs,targets) in enumerate(test_dl):
    yhat = model(inputs)
    #Retrieving predictions and true values as numpy arrays
    yhat = yhat.detach().numpy()
    actual = targets.numpy()
    actual = actual.reshape((len(actual),1))
    yhat = yhat.round() #Gives nearest integer class values as predictions
    # For multi-class problems, conversion of predictions to class values is as follows:
      # yhat = argmax(yhat, axis=1)
      # yhat = yhat.reshape((len(yhat), 1))
    predictions.append(yhat)
    actuals.append(actual)
  predictions, actuals = vstack(predictions), vstack(actuals)
  #Calculating the accuracy
  acc = accuracy_score(actuals, predictions)
  return acc

In [120]:
acc = evaluate_model(test_dl, model)
print("Accuracy: {}".format(acc))

Accuracy: 0.8879310344827587


### Making predictions

In [121]:
from torch import Tensor

In [122]:
# Make predictions

def predict(row, model):
  row = Tensor([row])
  yhat = model(row)
  yhat = yhat.detach().numpy()
  return yhat

In [123]:
# Make a single prediction (Expected class = 1)

row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print("Predicted value: {} (i.e Class = {})".format(yhat,yhat.round()))

Predicted value: [[0.9994531]] (i.e Class = [[1.]])
